# Kaggle project

Analyzing weather data and GIS data and predicting whether or not West Nile virus is present, for a given time, location, and species

In [176]:
import numpy as np
import pandas as pd

In [177]:
# load datasets

weather = pd.read_csv('assets/weather.csv')
spray = pd.read_csv('assets/spray.csv')
train = pd.read_csv('assets/train.csv')
test = pd.read_csv('assets/test.csv')

In train and test data:
- NumMosquitos: number of mosquitoes caught in this trap
- WnvPresent: whether West Nile Virus was present in these mosquitos. 1 means WNV is present, and 0 means not present. 


## Join weather and spray info to a new training dataset

In [178]:
train.head(2)

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0


In [179]:
train.shape

(10506, 12)

In [180]:
def r(x):
    return round(x * 1000)

sl = zip(spray.Latitude.map(r), spray.Longitude.map(r))
rl = zip(train.Latitude.map(r), train.Longitude.map(r))

train['ll_loc'] = zip(train.Latitude.map(r), train.Longitude.map(r))

def check(x):
    if x in sl:
        return False
    else:
        return True

train = train[train.ll_loc.map(check)]

In [181]:
# Create a column which identifies which weather station is closest to the trap

#Station 1: CHICAGO O'HARE INTERNATIONAL AIRPORT Lat: 41.995 Lon: -87.933 Elev: 662 ft. above sea level
#Station 2: CHICAGO MIDWAY INTL ARPT Lat: 41.786 Lon: -87.752 Elev: 612 ft. above sea level

station_1_lat = 41.995
station_1_long = -87.933
station_2_lat = 41.786
station_2_long = -87.752

# Combine lat & long into the one column

train['lat_long_comb'] = zip(train['Latitude'],train['Longitude'])

# Function to calculate distance from trap to each of the weather stations and then which is closest

def station(value):
    lat, lon = value
    dist_1 = np.sqrt(abs(station_1_lat - lat) + abs(station_1_long - lon))
    dist_2 = np.sqrt(abs(station_2_lat - lat) + abs(station_2_long - lon))
    if dist_1 > dist_2:
        return 2
    else:
        return 1

train['closest_station'] = train['lat_long_comb'].apply(station)

In [182]:
train.shape

(9543, 15)

In [183]:
cols= ['Station', 'Date', 'Tavg', 'DewPoint', 'WetBulb', 
       'Heat', 'Cool', 'StnPressure', 'SeaLevel', 'ResultSpeed', 'AvgSpeed']
weather_chosen = weather[cols]

In [184]:
def checkifM(x):
    if x == 'M':
        return np.nan
    else:
        return x
        
weather_chosen = weather_chosen.applymap(checkifM)
weather_chosen.isnull().sum()

Station         0
Date            0
Tavg           11
DewPoint        0
WetBulb         4
Heat           11
Cool           11
StnPressure     4
SeaLevel        9
ResultSpeed     0
AvgSpeed        3
dtype: int64

In [185]:
weather_chosen = weather_chosen.fillna(method='pad')

In [186]:
w_1 = weather_chosen[weather_chosen.Station == 1]
w_2 = weather_chosen[weather_chosen.Station == 2]

In [187]:
t = train.copy()
t.drop(['Address', 'AddressNumberAndStreet', 'll_loc', 'lat_long_comb', 'Street'], axis=1, inplace=True)

In [196]:
t.Date.unique()

array(['2007-05-29', '2007-06-05', '2007-06-26', '2007-06-29',
       '2007-07-02', '2007-07-11', '2007-07-18', '2007-07-19',
       '2007-07-25', '2007-07-27', '2007-08-01', '2007-08-02',
       '2007-08-03', '2007-08-07', '2007-08-08', '2007-08-09',
       '2007-08-15', '2007-08-16', '2007-08-17', '2007-08-21',
       '2007-08-22', '2007-08-24', '2007-08-28', '2007-09-04',
       '2007-09-05', '2007-09-06', '2007-09-12', '2007-09-18',
       '2007-09-19', '2007-09-24', '2007-09-25', '2007-10-04',
       '2007-10-09', '2009-05-28', '2009-06-02', '2009-06-03',
       '2009-06-05', '2009-06-12', '2009-06-15', '2009-06-19',
       '2009-06-22', '2009-06-26', '2009-06-29', '2009-07-06',
       '2009-07-10', '2009-07-13', '2009-07-17', '2009-07-24',
       '2009-07-27', '2009-07-31', '2009-08-07', '2009-08-13',
       '2009-08-25', '2009-08-27', '2009-09-03', '2009-09-14',
       '2009-09-17', '2009-09-25', '2009-10-01', '2011-06-10',
       '2011-06-17', '2011-06-24', '2011-06-30', '2011-

In [218]:
dat = '2007-05-29'
st = 1

i = w_1[w_1.Date == dat].index[0] 
print range(i, i+14, 2)

[56, 58, 60, 62, 64, 66, 68]


## EDA

## Build model

## Get result